In [1]:
import tensorflow as tf
import numpy as np 
import os
import time

In [2]:
input_layer = tf.placeholder(shape=(None,224,224,3), dtype="float32", name="input")

In [3]:
L1 = tf.layers.Conv2D(filters=64, kernel_size=7, strides=2)(input_layer)
L2 = tf.layers.BatchNormalization()(L1)
L3 = tf.nn.relu(L2)
L4 = tf.layers.MaxPooling2D(pool_size=3, strides=2)(L3)

In [4]:
def conv_block(input_layer, kernel, filters, stage, block, strides):
    
    filter1, filter2, filter3 = filters
    
    conv_name_base = 'res_'+str(stage)+"_conv_block_"+str(block)+"_branch"
    bn_base_name = 'bn_'+str(stage)+"_conv_block_"+str(block)+"_branch"
    relu_base_name = 'relu_'+str(stage)+"_conv_block_"+str(block)+"_branch"
    
    x = tf.layers.Conv2D(filters=filter1, kernel_size=1, strides=strides, name=conv_name_base+"_2a")(input_layer)
    x = tf.layers.BatchNormalization(name=bn_base_name+"_2a")(x)
    x = tf.nn.relu(x, name=relu_base_name+"_2a")
    
    x = tf.layers.Conv2D(filters=filter2, kernel_size=kernel, strides=strides, name=conv_name_base+"_2b")(x)
    x = tf.layers.BatchNormalization(name=bn_base_name+"_2b")(x)
    x = tf.nn.relu(x, name=relu_base_name+"_2b")    
    
    x = tf.layers.Conv2D(filters=filter3, kernel_size=1, strides=strides, name=conv_name_base+"_2c")(x)
    x = tf.layers.BatchNormalization(name=bn_base_name+"_2c")(x)    
    
    shortcut = tf.layers.Conv2D(filters=filter3, kernel_size=1, strides=strides, name=conv_name_base+"_1")(x)
    shortcut = tf.layers.BatchNormalization(name=bn_base_name+"_1")(shortcut)
    
    x = tf.add(x, shortcut)
    x = tf.nn.relu(x,name=relu_base_name+"_2b_1")
    
    return x
    

In [5]:
def identity_block(input_layer, kernel, filters, stage, block, strides):
    
    filter1, filter2, filter3 = filters
    
    conv_name_base = 'res_'+str(stage)+"_identity_block_"+str(block)+"_branch"
    bn_base_name = 'bn_'+str(stage)+"_identity_block_"+str(block)+"_branch"
    relu_base_name = 'relu_'+str(stage)+"_identity_block_"+str(block)+"_branch"
    
    x = tf.layers.Conv2D(filters=filter1, kernel_size=1, strides=strides, name=conv_name_base+"_2a")(input_layer)
    x = tf.layers.BatchNormalization(name=bn_base_name+"_2a")(x)
    x = tf.nn.relu(x, name=relu_base_name+"_2a")
    
    x = tf.layers.Conv2D(filters=filter2, kernel_size=kernel, strides=strides, name=conv_name_base+"_2b")(x)
    x = tf.layers.BatchNormalization(name=bn_base_name+"_2b")(x)
    x = tf.nn.relu(x, name=relu_base_name+"_2b")    
    
    x = tf.layers.Conv2D(filters=filter3, kernel_size=1, strides=strides, name=conv_name_base+"_2c")(x)
    x = tf.layers.BatchNormalization(name=bn_base_name+"_2c")(x)    
    
    x = tf.add(x, input_layer)
    x = tf.nn.relu(x,name=relu_base_name+"_2b_1")
    
    return x
    

In [6]:
x = conv_block(L4, kernel=1, filters=[64,64,256], stage=2, block="a", strides=1)
x = identity_block(x, 3, [64,64,256], stage=2, block="b", strides=1)
x = identity_block(x, 3, [64,64,256], stage=2, block="c", strides=1)

ValueError: Dimensions must be equal, but are 52 and 54 for 'Add_1' (op: 'Add') with input shapes: [?,52,52,256], [?,54,54,256].

In [ ]:
x

In [ ]:
x = conv_block(x, kernel=3, filters=[128, 128, 512], stage=3, block="a", strides=2)
x = identity_block(x, 3, [128, 128, 512], stage=3, block="b", strides=1)
x = identity_block(x, 3, [128, 128, 512], stage=3, block="c", strides=1)
x = identity_block(x, 3, [128, 128, 512], stage=3, block="d", strides=1)

In [ ]:
x

In [ ]:
x = conv_block(x, kernel=3, filters=[256, 256, 1024], stage=4, block="a", strides=2)
x = identity_block(x, 3, [256, 256, 1024], stage=4, block="b", strides=1)
x = identity_block(x, 3, [256, 256, 1024], stage=4, block="c", strides=1)
x = identity_block(x, 3, [256, 256, 1024], stage=4, block="d", strides=1)
x = identity_block(x, 3, [256, 256, 1024], stage=4, block="e", strides=1)
x = identity_block(x, 3, [256, 256, 1024], stage=4, block="f", strides=1)

In [ ]:
x

In [ ]:
x = conv_block(x, kernel=3, filters=[512, 512, 2048], stage=5, block="a", strides=2)
x = identity_block(x, 3, [512, 512, 2048], stage=5, block="b", strides=1)
x = identity_block(x, 3, [512, 512, 2048], stage=5, block="c", strides=1)

In [ ]:
x

In [ ]:
x = tf.layers.AveragePooling2D(pool_size=7, strides=1)(x)
x = tf.layers.dense(inputs=x, units=1000)

In [ ]:
x

In [ ]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    print(variable.name, shape)
    print()
    variable_parameters = 1
    for dim in shape:
#         print(dim)
        variable_parameters *= dim.value
#     print(variable_parameters)
    total_parameters += variable_parameters
    print("Ever time : ", total_parameters)
print(total_parameters)

In [ ]:
sess = tf.Session()

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
# run_meta = tf.RunMetadata()
t1 = time.time()    
sess.run(x, feed_dict={input_layer : np.random.randn(1,224,224,3).astype("float32")})
print(time.time()-t1)#, options=run_options, run_metadata=run_meta)
# opts = tf.profiler.ProfileOptionBuilder.float_operation()
# flops = tf.profiler.profile(graph=sess.graph,run_meta=run_meta, cmd='op', options=opts)
# print(flops.total_float_ops)

In [ ]:
# saver = tf.train.Saver()
# saver.save(sess, save_path="./ResNet50")